#Set-Up

In [1]:
# Install necessary packages
%pip install highspy

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ------------------------------- -------- 1.6/2.0 MB 8.3 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 9.5 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo and Coin-OR solvers.
#for reference, see https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html#installing-pyomo-and-solvers

#%%capture
import sys
import os


if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *
from pyomo.contrib.appsi.solvers import Highs
# Quick check that the solver is available
solver = Highs()
print("Solver is available: ", solver.available())

Solver is available:  FullLicense


#Problem A: Practice writing an optimizing problem
A rectangular box must have volume 500 in^3 (Recall: a box's volume is defined by its $length*width*height$).

Find the shape that has the smallest "mailing length" where the mailing length is defined as the sum of the three edge lengths.

What is the optimal shape of the box that meets this volume requirement with the smallest mailing length possible? (Use ipopt since this is a nonlinear problem).

In [7]:
# Setup Model
# Minimize x1 + x2 + x3
# S.T. x1 * x2 * x3 = 500 in^3
#     x1, x2, x3, > 0
num_dv = 3

#initialize a "Concrete Model"
modelA = ConcreteModel()

#initialize DVs
modelA.x = Var(range(num_dv), domain = NonNegativeReals)

#define the objective
modelA.obj = Objective(expr = sum(modelA.x[i] for i in range(num_dv)), sense = minimize)
#define the constraint
modelA.constraint1 = Constraint(expr = modelA.x[0]*modelA.x[1]*modelA.x[2] == 500)
modelA.constraint2 = Constraint(expr = modelA.x[0] >= 0)
modelA.constraint3 = Constraint(expr = modelA.x[1] >= 0)
modelA.constraint4 = Constraint(expr = modelA.x[2] >= 0)

#(Optional) You can use model.pprint() to see what you've done so far
modelA.pprint()

1 Var Declarations
    x : Size=3, Index={0, 1, 2}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] + x[1] + x[2]

4 Constraint Declarations
    constraint1 : Size=1, Index=None, Active=True
        Key  : Lower : Body           : Upper : Active
        None : 500.0 : x[0]*x[1]*x[2] : 500.0 :   True
    constraint2 : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :   0.0 : x[0] :  +Inf :   True
    constraint3 : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :   0.0 : x[1] :  +Inf :   True
    constraint4 : Size=1, Index=None, Ac

In [8]:
#solve model (Note: you should use "ipopt" because this is a nonlinear problem)
opt = SolverFactory('ipopt')
results = opt.solve(modelA, tee = False) #setting tee = False hides the diagnostic outputs


In [11]:
#print optimal solution and the smallest mailing length it achieves
for i in range(num_dv):
  print(f'x{i+1} = {round(modelA.x[i].value, 2)}')
print(f'optimal mailing length = {round(modelA.x[0].value + modelA.x[1].value + modelA.x[2].value,2)}')

x1 = 7.94
x2 = 7.94
x3 = 7.94
optimal mailing length = 23.81


#Problem B: Optimizing an Existing Function (office building)
Below, I've included a completed "office building" model as a function. Use Pyomo to solve for the price per square foot in each year that maximizes the total earnings after tax. Use ipopt (since this is a nonlinear problem).

In [4]:
def office_earnings(total_sqft = 180000,
           m = -0.05,
           b = 1.5,
           op_expense_per_sqft = 1.20,
           heating_surcharge_per_sqft = .2,
           op_exp_annual_growth = .12,
           annual_mortgage = 1500000,
           tax_rate = .34,
           price_per_sqft = [15, 15, 15, 15, 15],
           num_years = 5):
  #rev calc
  perc_occ = [m*price_per_sqft[i] + b for i in range(num_years)]
  sqft_occ = [perc_occ[i]*total_sqft for i in range(num_years)]
  revenue = [sqft_occ[i]*price_per_sqft[i] for i in range(num_years)]
  #operating expense calculations
  base_op_cost_as_percY1 = [(1+op_exp_annual_growth)**i for i in range(num_years)] #note that range(num_years) = range(5) = [0,1,2,3,4] and (1+op_exp_annual_growth)**0 = 1.
  base_op_cost = [op_expense_per_sqft*total_sqft*base_op_cost_as_percY1[i] for i in range(num_years)]
  heating_surcharge = [perc_occ[i]*base_op_cost[i]*heating_surcharge_per_sqft for i in range(num_years)]
  mortgage = [annual_mortgage for i in range(num_years)]
  operating_costs = [base_op_cost[i] + heating_surcharge[i] + mortgage[i] for i in range(num_years)]
  #before and after-tax earnings
  ebt = [revenue[i] - operating_costs[i] for i in range(num_years)]
  taxes = [ebt[i]*tax_rate for i in range(num_years)]
  earnings_after_tax = [ebt[i] - taxes[i] for i in range(num_years)]
  total_earnings_after_tax = sum(earnings_after_tax)
  return total_earnings_after_tax

In [5]:
#initialize a "Concrete Model"
modelB = ConcreteModel()

#initialize DVs
modelB.price_per_sqft_dv = Var(range(5), domain = NonNegativeReals)

#define the objective
modelB.obj = Objective(expr = office_earnings(price_per_sqft = modelB.price_per_sqft_dv), sense = maximize)

#(Optional) You can use model.pprint() to see what you've done so far
modelB.pprint()

1 Var Declarations
    price_per_sqft_dv : Size=5, Index={0, 1, 2, 3, 4}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals
          4 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : (-0.05*price_per_sqft_dv[0] + 1.5)*180000*price_per_sqft_dv[0] - (216000.0 + (-0.05*price_per_sqft_dv[0] + 1.5)*216000.0*0.2 + 1500000) - ((-0.05*price_per_sqft_dv[0] + 1.5)*180000*price_per_sqft_dv[0] - (216000.0 + (-0.05*price_per_sqft_dv[0] + 1.5)*216000.0*0.2 + 1500000))*0.34 + (-0.05*price_per_sqft_dv[1] + 1.5)*180000*price_per_sqft_dv[1] - (241920.000000

In [18]:
#solve model (Note: you should use "ipopt" because this is a nonlinear problem)
opt = SolverFactory('ipopt')
results = opt.solve(modelB, tee = False) #setting tee = False hides the diagnostic outputs


In [20]:
#print optimal solution and the largest earnings it achieves
for i in range(5):
  print(f'price per sqft in year {i+1} = {round(modelB.price_per_sqft_dv[i].value, 2)}')
  print(f'earnings after tax in year {i+1} = {round(office_earnings(price_per_sqft = [round(modelB.price_per_sqft_dv[i].value, 2) for i in range(5)]), 2)}')
print(f"Total earnings after tax = {round(modelB.obj(), 2)}")

price per sqft in year 1 = 15.12
earnings after tax in year 1 = 691696.7
price per sqft in year 2 = 15.13
earnings after tax in year 2 = 691696.7
price per sqft in year 3 = 15.15
earnings after tax in year 3 = 691696.7
price per sqft in year 4 = 15.17
earnings after tax in year 4 = 691696.7
price per sqft in year 5 = 15.19
earnings after tax in year 5 = 691696.7
Total earnings after tax = 691696.83


#Problem C: Coding with Lists of Lists and Constraint Lists
Solve this small version of the Stigler problem shown below using lists, `ConstraintLists`, and `for` loops. This version only has **4 decision variables (DVs)** and **3 constraints** but please code in a way that would be scalable for larger problems by following the structure I've started for you below. Print out the optimal \(x\)'s and the total optimal cost.

**Minimize cost =**  
$0.36*x_{\text{wheat}} + 0.141*x_{\text{mac}} + 0.242*x_{\text{cereal}} + 0.300*x_{\text{milk}}$

**Subject to:**  
$
16.1*x_{\text{wheat}} + 1.6*x_{\text{mac}} + 2.9*x_{\text{cereal}} + 12.5*x_{\text{milk}} \geq 3 \quad (\text{Calories Daily Min Constraint})
$
$
7.9*x_{\text{wheat}} + 58.9*x_{\text{mac}} + 91.2*x_{\text{cereal}} + 42.3*x_{\text{milk}} \geq 1.8 \quad (\text{Protein Daily Min Constraint})
$
$
80.5*x_{\text{wheat}} + 3.0*x_{\text{mac}} + 7.2*x_{\text{cereal}} + 15.4*x_{\text{milk}} \geq 2.5 \quad (\text{Fiber Daily Min Constraint})
$

$
x_{\text{wheat}}, x_{\text{mac}}, x_{\text{cereal}}, x_{\text{milk}} \geq 0
$


In [6]:
#I've put in these input parameters for you
num_commodities = 4    #this is how many food commodities to decide on
num_nutrients = 3      #this is how many nutrient constraints there are
cost_coef = [.36, 0.141, 0.242, 0.300]   #this is a list of the cost coefficients in the objective
constraint_coef = [[16.1, 1.6, 2.9, 12.5],
                   [7.9, 58.9, 91.2, 42.3],
                   [80.5, 3.0, 7.2, 15.4]]     #this is the list of lists of all the constraint coefficients
daily_mins = [3, 1.8, 2.5]        #these are the right hand sides

In [7]:
#Fill in the ??? to complete this code
modelC = ConcreteModel()

#dvs
modelC.x = Var(range(num_commodities), domain = NonNegativeReals)

#objective
modelC.Objective = Objective(expr = sum(cost_coef[i] * modelC.x[i] for i in range(num_commodities)), sense = minimize)

#constraints
modelC.nutrient_constraints = ConstraintList()
for i in range(num_nutrients):
  modelC.nutrient_constraints.add(expr = sum(constraint_coef[i][j] * modelC.x[j] for j in range(num_commodities)) >= daily_mins[i])

#model pprint()
modelC.pprint()

1 Var Declarations
    x : Size=4, Index={0, 1, 2, 3}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
          3 :     0 :  None :  None : False :  True : NonNegativeReals

1 Objective Declarations
    Objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : 0.36*x[0] + 0.141*x[1] + 0.242*x[2] + 0.3*x[3]

1 Constraint Declarations
    nutrient_constraints : Size=3, Index={1, 2, 3}, Active=True
        Key : Lower : Body                                         : Upper : Active
          1 :   3.0 :  16.1*x[0] + 1.6*x[1] + 2.9*x[2] + 12.5*x[3] :  +Inf :   True
          2 :   1.8 : 7.9*x[0] + 58.9*x[1] + 91.2*x[2] + 42.3*x[3] :  +Inf :   True
          3 :   2.5 :  80.5*x[0] + 3.0*x[1] + 7.2*x[2] +

In [11]:
#solve model with cbc because this is a linear program
opt = Highs()
results = opt.solve(modelC)

In [13]:
# print solver status and optimal results
#print("Solver Status:", results.solver.status)
#print("Termination Condition:", results.solver.termination_condition)
#print("----------------------------------------------------")

#print optimal amounts of each food and the optimal cost it achieves
for i in range(num_commodities):
  print(f'x{i+1} = {round(modelC.x[i].value, 2)}')
print(f'optimal cost = {round(modelC.Objective(), 2)}')


x1 = 0.18
x2 = 0.0
x3 = 0.0
x4 = 0.01
optimal cost = 0.07
